<a href="https://colab.research.google.com/github/Krism-T/TRIAL-RF/blob/main/SKG_RANDOM_FOREST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE

import joblib

In [2]:
!pip install imbalanced-learn streamlit pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 21.2 MB/s eta 0:00:00


In [3]:
# A. LOAD DATA
# =========================
sheet_url = 'https://docs.google.com/spreadsheets/d/1AfcS9SYlAba88BgWKhOIyKNR8TMlh379wvi8RlklVCI/edit?usp=sharing'
csv_url = sheet_url.replace('/edit?usp=sharing', '/export?format=csv')

df = pd.read_csv(csv_url)
print(f"Dataframe berhasil dimuat. Total baris: {len(df)}")

# Bersihkan NA
df = df.dropna()
print(f"Total baris setelah drop NA: {len(df)}")

Dataframe berhasil dimuat. Total baris: 72
Total baris setelah drop NA: 72


In [4]:
# B. PREPROCESSING
# =========================
TARGET_COLUMN = 'LevelKompetensi'

# Semua variabel fitur sudah numerik → tidak perlu OHE
# Pilih semua kolom kecuali target & ID sebagai fitur
feature_cols = [c for c in df.columns if c not in [TARGET_COLUMN, 'ID']]
X = df[feature_cols].copy()

# Pastikan benar-benar numerik (opsional, kalau sudah yakin boleh di-skip)
X = X.apply(pd.to_numeric, errors='raise')

# Encode target
le = LabelEncoder()
df['y_encoded'] = le.fit_transform(df[TARGET_COLUMN])
y = df['y_encoded']

print("Mapping target (kode → label):")
for code, label in enumerate(le.classes_):
    print(code, ":", label)

# Tentukan kelas yang akan di-SMOTE: "Novice Teacher" dan "Expert Teacher"
target_smote_labels = ['Novice Teacher', 'Expert Teacher']
smote_classes = le.transform(target_smote_labels)  # ini dalam bentuk kode numerik
print("\nKelas yang akan di-oversample dengan SMOTE:")
for lbl, code in zip(target_smote_labels, smote_classes):
    print(f"{lbl} -> kode {code}")

# split train-test (tetap stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Ukuran X_train:", X_train.shape)
print("Ukuran X_test :", X_test.shape)


Mapping target (kode → label):
0 : Experienced Teacher
1 : Expert Teacher
2 : Novice Teacher

Kelas yang akan di-oversample dengan SMOTE:
Novice Teacher -> kode 2
Expert Teacher -> kode 1
Ukuran X_train: (57, 13)
Ukuran X_test : (15, 13)


In [5]:
from collections import Counter
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder

# ... (bagian load data & preprocessing kamu tetap sama)

TARGET_COLUMN = 'LevelKompetensi'

# fitur numerik (semua sudah numerik)
feature_cols = [c for c in df.columns if c not in [TARGET_COLUMN, 'ID']]
X = df[feature_cols].copy()
X = X.apply(pd.to_numeric, errors='raise')

# encode target
le = LabelEncoder()
df['y_encoded'] = le.fit_transform(df[TARGET_COLUMN])
y = df['y_encoded']

print("Mapping target (kode → label):")
for code, label in enumerate(le.classes_):
    print(code, ":", label)

# ambil kode kelas untuk "Novice Teacher" dan "Expert Teacher"
target_smote_labels = ['Novice Teacher', 'Expert Teacher']
smote_classes = le.transform(target_smote_labels)
print("\nKelas yang akan di-oversample dengan SMOTE:")
for lbl, code in zip(target_smote_labels, smote_classes):
    print(f"{lbl} -> kode {code}")

# split train-test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Ukuran X_train:", X_train.shape)
print("Ukuran X_test :", X_test.shape)

# Determine global target count for minority classes based on majority class in full X_train
train_class_counts = Counter(y_train)
majority_class_code = train_class_counts.most_common(1)[0][0] # Get the code of the majority class
target_count_for_minority = train_class_counts[majority_class_code] # Get its count

# ============ FUNGSI STRATEGI SMOTE BARU ============== #
def smote_strategy(y_array):
    """
    y_array: array label di fold tertentu.
    Mengembalikan dict: {kelas: target_jumlah_sampel}
    Kelas minoritas ('Novice Teacher', 'Expert Teacher') akan di-oversample
    hingga target_count_for_minority.
    """
    counter = Counter(y_array)
    strategy = {}

    for cls_code in smote_classes: # Iterate through Novice and Expert Teacher codes
        if cls_code in counter:
            # Oversample to target_count_for_minority if current count is less
            if counter[cls_code] < target_count_for_minority:
                strategy[cls_code] = target_count_for_minority
            else:
                strategy[cls_code] = counter[cls_code] # If already at or above target, keep current count
        # If class not in fold, it cannot be oversampled in this specific fold.
        # So, we don't add it to the strategy dictionary.

    # Ensure majority classes not explicitly targeted for oversampling are kept at their original count
    for cls_code, count in counter.items():
        if cls_code not in smote_classes:
            strategy[cls_code] = count # Keep majority class count as is

    return strategy

# =========================
# C. PIPELINE + GRIDSEARCH
# =========================
pipeline = ImbPipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(
        random_state=42,
        k_neighbors=1, # Reduced k_neighbors to be safer for small classes
        sampling_strategy=smote_strategy
    )),
    ('rf', RandomForestClassifier(random_state=42))
])

param_grid = {
    'rf__n_estimators': [100, 200],
    'rf__max_depth': [10, 20, None],
    'rf__min_samples_leaf': [1, 2]
}

cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=cv_strategy,
    scoring='f1_weighted',
    n_jobs=-1,
    verbose=1
)

print("\nMemulai GridSearch...")
grid_search.fit(X_train, y_train)

best_pipeline = grid_search.best_estimator_
print(f"\nSkor F1-weighted terbaik (CV): {grid_search.best_score_:.4f}")
print("Best params:", grid_search.best_params_)


Mapping target (kode → label):
0 : Experienced Teacher
1 : Expert Teacher
2 : Novice Teacher

Kelas yang akan di-oversample dengan SMOTE:
Novice Teacher -> kode 2
Expert Teacher -> kode 1
Ukuran X_train: (57, 14)
Ukuran X_test : (15, 14)

Memulai GridSearch...
Fitting 5 folds for each of 12 candidates, totalling 60 fits

Skor F1-weighted terbaik (CV): 0.9583
Best params: {'rf__max_depth': 10, 'rf__min_samples_leaf': 1, 'rf__n_estimators': 100}


In [6]:
# D. EVALUASI
# =========================
y_pred = best_pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"\n>>> AKURASI TEST: {accuracy:.4f}\n")
print("--- Laporan Klasifikasi ---")
print(classification_report(y_test, y_pred, target_names=le.classes_))

cm = confusion_matrix(y_test, y_pred)
print("\n--- Confusion Matrix ---")
print(pd.DataFrame(cm, index=le.classes_, columns=le.classes_))

# =========================
# E. SIMPAN MODEL + METADATA
# =========================
joblib.dump(best_pipeline, 'random_forest_pipeline.joblib')
joblib.dump(le, 'label_encoder.joblib')
joblib.dump(X.columns.tolist(), 'model_features.joblib')

print("\n✅ Model, label encoder, dan daftar fitur berhasil disimpan.")


>>> AKURASI TEST: 0.9333

--- Laporan Klasifikasi ---
                     precision    recall  f1-score   support

Experienced Teacher       1.00      1.00      1.00        10
     Expert Teacher       0.80      1.00      0.89         4
     Novice Teacher       0.00      0.00      0.00         1

           accuracy                           0.93        15
          macro avg       0.60      0.67      0.63        15
       weighted avg       0.88      0.93      0.90        15


--- Confusion Matrix ---
                     Experienced Teacher  Expert Teacher  Novice Teacher
Experienced Teacher                   10               0               0
Expert Teacher                         0               4               0
Novice Teacher                         0               1               0

✅ Model, label encoder, dan daftar fitur berhasil disimpan.


In [7]:
%%writefile app_streamlit.py
import streamlit as st
import pandas as pd
import joblib

# ============================
# 1. BACA MAPPING DARI EXCEL
# ============================
@st.cache_data
def load_mapping_from_excel(path: str = "MAPPING.xlsx"):
    df_map = pd.read_excel(path)

    required_cols = {"Variabel", "Code", "Meaning"}
    if not required_cols.issubset(set(df_map.columns)):
        raise ValueError(f"Kolom di MAPPING.xlsx harus mengandung: {required_cols}")

    mapping = {}
    for var in df_map["Variabel"].unique():
        sub = df_map[df_map["Variabel"] == var]
        mapping[var] = {
            str(row["Meaning"]): int(row["Code"])
            for _, row in sub.iterrows()
        }
    return mapping

try:
    MAPPING = load_mapping_from_excel("MAPPING.xlsx")
except Exception as e:
    st.error(f"Gagal memuat MAPPING.xlsx: {e}")
    st.stop()

categorical_features = list(MAPPING.keys())

Writing app_streamlit.py


In [8]:
# 2. LOAD MODEL & METADATA
# ============================
try:
    model_pipeline = joblib.load("random_forest_pipeline.joblib")
    le = joblib.load("label_encoder.joblib")
    model_features = joblib.load("model_features.joblib")
except Exception as e:
    st.error(f"Gagal memuat model atau metadata (.joblib): {e}")
    st.stop()

In [9]:
%%writefile app_streamlit.py
import streamlit as st
import pandas as pd
import joblib
from collections import Counter

# ============================
# 1. BACA MAPPING DARI EXCEL
# ============================
@st.cache_data
def load_mapping_from_excel(path: str = "MAPPING.xlsx"):
    df_map = pd.read_excel(path)

    required_cols = {"Variabel", "Code", "Meaning"}
    if not required_cols.issubset(set(df_map.columns)):
        raise ValueError(f"Kolom di MAPPING.xlsx harus mengandung: {required_cols}")

    mapping = {}
    for var in df_map["Variabel"].unique():
        sub = df_map[df_map["Variabel"] == var]
        mapping[var] = {
            str(row["Meaning"]): int(row["Code"])
            for _, row in sub.iterrows()
        }
    return mapping

try:
    MAPPING = load_mapping_from_excel("MAPPING.xlsx")
except Exception as e:
    st.error(f"Gagal memuat MAPPING.xlsx: {e}")
    st.stop()

categorical_features = list(MAPPING.keys())

# ============================
# 2. LOAD MODEL & METADATA
# ============================
try:
    # Load label encoder first, as it's needed for smote_classes
    le = joblib.load("label_encoder.joblib")

    # Define smote_classes and smote_strategy for model loading
    target_smote_labels = ['Novice Teacher', 'Expert Teacher']
    smote_classes = le.transform(target_smote_labels)

    def smote_strategy(y_array):
        counter = Counter(y_array)
        max_count = max(counter.values())
        strategy = {
            cls: max_count
            for cls in smote_classes
            if cls in counter
        }
        return strategy

    model_pipeline = joblib.load("random_forest_pipeline.joblib")
    model_features = joblib.load("model_features.joblib")

    # Load evaluation metrics
    model_accuracy = joblib.load('model_accuracy.joblib')
    model_classification_report = joblib.load('model_classification_report.joblib')
    model_confusion_matrix = joblib.load('model_confusion_matrix.joblib')
    model_roc_auc = joblib.load('model_roc_auc.joblib')

except Exception as e:
    st.error(f"Gagal memuat model atau metadata (.joblib): {e}")
    st.stop()

# ============================
# 3. UI STREAMLIT
# ============================
st.set_page_config(layout="wide")
st.title("👨‍🏫 Prediksi Level Kompetensi Guru (SKG 360)")

st.markdown(
    """
Aplikasi ini menggunakan model **Random Forest** untuk memprediksi level kompetensi guru berdasarkan data demografis/latar belakang guru.
"""
)

# Initialize session state for form key if not already present
if 'form_key_counter' not in st.session_state:
    st.session_state.form_key_counter = 0

# Function to increment the form key, effectively resetting the form
def reset_form():
    st.session_state.form_key_counter += 1

# Place the 'New Input' button outside the form for a full reset
# Add 'on_click' to trigger the reset_form function
st.button("Input Baru", on_click=reset_form)

def _options(var_name):
    opts = list(MAPPING.get(var_name, {}).keys())
    # Add a placeholder for an empty selection
    return ["--Pilih--"] + opts if len(opts) > 0 else ["--Pilih--", "(tidak ada data)"]

with st.form(key=f"my_form_{st.session_state.form_key_counter}"):
    st.subheader("Input Data Guru")
    col1, col2 = st.columns(2)

    with col1:
        # Remove default value, will default to min_value
        usia = st.number_input("Usia (tahun)", min_value=20, max_value=70)
        input_strings = {}
        input_strings["Sekolah"] = st.selectbox("Sekolah", _options("Sekolah"))
        input_strings["Departemen"] = st.selectbox("Departemen", _options("Departemen"))
        input_strings["Gender"] = st.selectbox("Gender", _options("Gender"))
        input_strings["Mapel"] = st.selectbox("Mata Pelajaran", _options("Mapel"))

    with col2:
        # Remove default value, will default to min_value
        pengalaman = st.number_input("Pengalaman Mengajar (tahun)", min_value=0, max_value=40)

        input_strings["Region Lahir"] = st.selectbox("Region Lahir", _options("Region Lahir"))
        input_strings["katalog.StatusKaryawanChoices"] = st.selectbox(
            "Status Karyawan", _options("katalog.StatusKaryawanChoices")
        )
        input_strings["katalog.MaritalStatus"] = st.selectbox(
            "Status Perkawinan", _options("katalog.MaritalStatus")
        )
        input_strings["TOEFL>400"] = st.selectbox(
            "Memiliki Sertifikat TOEFL ≥ 400?", _options("TOEFL>400")
        )
        input_strings["Pendidikan"] = st.selectbox(
            "Pendidikan Terakhir", _options("Pendidikan")
        )
        input_strings["Prodi"] = st.selectbox("Program Studi S1", _options("Prodi"))
        input_strings["UnivS1"] = st.selectbox("Universitas S1", _options("UnivS1"))

    submitted = st.form_submit_button("Prediksi Level Kompetensi")

if submitted:
    # Initialize a flag to check if we can proceed with prediction
    can_predict = True

    # ============================
    # 4. BANGUN DATAFRAME INPUT
    # ============================
    raw_data = {
        "katalog.Usia": usia,
        "Pengalaman": pengalaman
    }

    for kol, label_str in input_strings.items():
        # Validate that a selection has been made for selectbox inputs
        if label_str == "--Pilih--":
            st.error(f"Mohon pilih nilai untuk '{kol}'.")
            can_predict = False
            continue # Continue to check other inputs

        try:
            raw_data[kol] = MAPPING[kol][label_str]
        except KeyError:
            st.error(f"Nilai '{label_str}' tidak ditemukan di MAPPING untuk variabel '{kol}'.")
            can_predict = False
            continue # Continue to check other inputs

    # ============================
    # 5. OHE & ALIGN DENGAN MODEL
    # ============================
    if can_predict: # Only proceed if all validations passed
        input_df = pd.DataFrame([raw_data])
        input_encoded = pd.get_dummies(input_df, columns=categorical_features, drop_first=True)

        final_input = pd.DataFrame(columns=model_features).fillna(0)
        for col in input_encoded.columns:
            if col in final_input.columns:
                final_input[col] = input_encoded[col]

        final_input = final_input.fillna(0)
        final_input = final_input[model_features]

        # ============================
        # 6. PREDIKSI
        # ============================
        try:
            pred_encoded = model_pipeline.predict(final_input)
            pred_label = le.inverse_transform(pred_encoded)[0]

            proba = None
            try:
                proba = model_pipeline.predict_proba(final_input)[0]
            except:
                pass

            st.success(f"### Hasil Prediksi Level Kompetensi: **{pred_label}**")

            if proba is not None:
                st.markdown("**Probabilitas tiap level:**")
                prob_df = pd.DataFrame({
                    "LevelKompetensi": le.classes_,
                    "Probabilitas": proba
                }).sort_values("Probabilitas", ascending=False)
                st.dataframe(prob_df, use_container_width=True)

            if "Expert" in pred_label:
                st.balloons()

        except Exception as e:
            st.error(f"Terjadi error saat prediksi: {e}")

# ============================
# 7. TAMPILKAN METRIK EVALUASI
# ============================
st.markdown("--- ")
st.subheader("Evaluasi Performa Model (pada Test Set)")

expander_metrics = st.expander("Lihat Metrik Evaluasi Model")
with expander_metrics:
    st.write(f"**Akurasi**: {model_accuracy:.4f}")
    st.write(f"**ROC-AUC (weighted)**: {model_roc_auc:.4f}")
    st.markdown("**Classification Report:**")
    st.code(model_classification_report, language='text')
    st.markdown("**Confusion Matrix:**")
    st.dataframe(model_confusion_matrix, use_container_width=True)


Overwriting app_streamlit.py


In [10]:
from pyngrok import ngrok
import subprocess, time
import os

# isi tokenmu di sini
ngrok.set_auth_token("36iFdHX5BlJ6rMGCg5TlRKDZY5B_3qaXrb3oqxKnwnakdPgfp")  # ganti ya

current_streamlit_proc = None

def stop_streamlit():
    """Matikan Streamlit & ngrok (jika ada)."""
    global current_streamlit_proc
    if current_streamlit_proc is not None:
        try:
            current_streamlit_proc.terminate()
            current_streamlit_proc.wait(timeout=5)
        except:
            try:
                current_streamlit_proc.kill()
            except:
                pass
        current_streamlit_proc = None

    try:
        ngrok.kill()
    except:
        pass

    os.system("fuser -k 8501/tcp >/dev/null 2>&1")
    print("✅ Streamlit & ngrok (jika ada) sudah dimatikan.")

def start_streamlit():
    """Stop instance lama, lalu jalankan Streamlit + ngrok baru."""
    global current_streamlit_proc

    stop_streamlit()

    print("🚀 Menjalankan Streamlit...")
    proc = subprocess.Popen(
        ["streamlit", "run", "app_streamlit.py",
         "--server.port", "8501", "--server.headless", "true"],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )
    current_streamlit_proc = proc

    time.sleep(10)

    print("🌐 Membuat tunnel ngrok...")
    tunnel = ngrok.connect(8501, bind_tls=True)

    print("✅ Aplikasi Streamlit siap diakses di URL:")
    print(tunnel.public_url)
    return tunnel.public_url


In [11]:
##RE-READ FILE MAPPING
import requests

sheet_url = 'https://docs.google.com/spreadsheets/d/1K74HlgKj19djz9EUeg0ZUIWMzM9Kt4TvS0oPn_CDsSc/edit?usp=sharing'
excel_export_url = sheet_url.replace('/edit?usp=sharing', '/export?format=xlsx')

file_name = 'MAPPING.xlsx'

try:
    response = requests.get(excel_export_url)
    response.raise_for_status() # Raise an exception for bad status codes

    with open(file_name, 'wb') as f:
        f.write(response.content)

    print(f"File '{file_name}' berhasil diunduh dan disimpan.")
except requests.exceptions.RequestException as e:
    print(f"Gagal mengunduh file: {e}")
except Exception as e:
    print(f"Terjadi kesalahan: {e}")

File 'MAPPING.xlsx' berhasil diunduh dan disimpan.


In [12]:
url = start_streamlit()
url

✅ Streamlit & ngrok (jika ada) sudah dimatikan.
🚀 Menjalankan Streamlit...
🌐 Membuat tunnel ngrok...
✅ Aplikasi Streamlit siap diakses di URL:
https://zona-squirelike-diedra.ngrok-free.dev


'https://zona-squirelike-diedra.ngrok-free.dev'

In [13]:
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Assuming y_test and best_pipeline are available from previous cells
# For multi-class, we need to handle ROC-AUC differently, e.g., 'ovr' or 'ovo'
# We'll use 'ovr' (One-vs-Rest) strategy and average='weighted'

y_pred_proba = best_pipeline.predict_proba(X_test)

# Calculate ROC-AUC score
# For multi-class, we calculate the ROC AUC for each class and then average them.
# We'll use 'ovr' (One-vs-Rest) strategy and average='weighted'
roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr', average='weighted')

print(f"\n>>> ROC-AUC Score (weighted, One-vs-Rest): {roc_auc:.4f}")

print(f"\n>>> AKURASI TEST: {accuracy:.4f}\n")
print("--- Laporan Klasifikasi ---")
classification_rep_str = classification_report(y_test, y_pred, target_names=le.classes_)
print(classification_rep_str)

cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=le.classes_, columns=le.classes_)
print("\n--- Confusion Matrix ---")
print(cm_df)

# Save evaluation metrics
joblib.dump(accuracy, 'model_accuracy.joblib')
joblib.dump(classification_rep_str, 'model_classification_report.joblib')
joblib.dump(cm_df, 'model_confusion_matrix.joblib')
joblib.dump(roc_auc, 'model_roc_auc.joblib')

print("\n✅ Metrik evaluasi model berhasil disimpan.")


>>> ROC-AUC Score (weighted, One-vs-Rest): 1.0000

>>> AKURASI TEST: 0.9333

--- Laporan Klasifikasi ---
                     precision    recall  f1-score   support

Experienced Teacher       1.00      1.00      1.00        10
     Expert Teacher       0.80      1.00      0.89         4
     Novice Teacher       0.00      0.00      0.00         1

           accuracy                           0.93        15
          macro avg       0.60      0.67      0.63        15
       weighted avg       0.88      0.93      0.90        15


--- Confusion Matrix ---
                     Experienced Teacher  Expert Teacher  Novice Teacher
Experienced Teacher                   10               0               0
Expert Teacher                         0               4               0
Novice Teacher                         0               1               0

✅ Metrik evaluasi model berhasil disimpan.
